In [1]:
import glob
from experiment import ExperimentEvaluation
import pandas as pd
from IPython.display import display
import itertools
import matplotlib.pyplot as plt
plt.style.use('seaborn')

d:\Avkar\AutoCodeReview\Code\Github-SourceCode-Review\v3\env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Kaan\AppData\Local\Temp\ipykernel_4448\3399956894.py:7: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [2]:
evaluation_paths = glob.glob(".\\results/*_C10*")

In [3]:
metric_results = []
for evaluation_path in evaluation_paths:
    exp_name, res = ExperimentEvaluation.load_results(evaluation_path)
    vectorizer, vec_dist, text_sim, candidate_count = exp_name.split("_")[:4]
    # candidate_count = int(candidate_count.replace("C", ""))
    keys = list(res.keys())
    metric_results.append({"vectorizer": vectorizer, "vec_dist": vec_dist, "text_sim": text_sim} | {key: res[key] for key in keys})

df = pd.DataFrame(metric_results)
df["exact_match_ratio"] = df.apply(lambda row: row["exact_matches"][1], axis=1)
# df["exact_match_count"] = df.apply(lambda row: row["exact_matches"][0], axis=1)

In [4]:
df.rename(columns={name: "ranged_means" for name in df.columns[df.columns.str.contains("ranged_means")]}, inplace=True)
df.rename(columns={name: "mean" for name in df.columns[df.columns.str.endswith("mean")]}, inplace=True)

In [5]:
def highlight_max(s):
    is_max = s == s.max()
    return ['color: green' if cell else '' for cell in is_max]

def highlight_min(s):
    is_max = s == s.min()
    return ['color: red' if cell else '' for cell in is_max]

In [6]:
def sjoin(x): return (x[x.notnull()])
df = df.groupby(level=0, axis=1, group_keys=False ).apply(lambda x: x.apply(sjoin, axis=1))

In [7]:
df_blue = df[(df["text_sim"] == "bleu") & (df["vec_dist"] != "cosine")]

In [8]:
blue_table = df_blue.pivot_table(index=["vectorizer"], columns="vec_dist")

C:\Users\Kaan\AppData\Local\Temp\ipykernel_4448\2786243585.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  blue_table = df_blue.pivot_table(index=["vectorizer"], columns="vec_dist")


In [9]:
metrics = blue_table.columns.get_level_values(0).unique()
for metric in metrics:
    # res_table[metric].style.apply(highlight_max, axis = 1).apply(highlight_min, axis = 1).set_caption(metric.title().replace("_", " ")).to_latex(f"latex/{metric}.tex")
    display(blue_table[metric].style.apply(highlight_max, axis = 0).apply(highlight_min, axis = 0).set_caption(metric.title().replace("_", " ")))

vec_dist,euclidean,manhattan
vectorizer,,
bert,3.700630,3.695985
bow,2.802020,2.928781
doc2vec,2.594954,2.561771
tfidf,2.992493,2.947363
word2vec-mean,2.825248,2.807329


vec_dist,euclidean,manhattan
vectorizer,,
bert,234.049442,1942.325117
bow,260.548046,284.080409
doc2vec,193.601840,1010.250785
tfidf,243.185203,258.211901
word2vec-mean,206.918270,1021.547677


vec_dist,euclidean,manhattan
vectorizer,,
bert,0.239011,0.238998
bow,0.219550,0.219713
doc2vec,0.216995,0.216344
tfidf,0.225273,0.224323
word2vec-mean,0.220623,0.220430


In [10]:
res_table = df.pivot_table(index=["vec_dist", "text_sim"], columns="vectorizer")

C:\Users\Kaan\AppData\Local\Temp\ipykernel_4448\2211323937.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  res_table = df.pivot_table(index=["vec_dist", "text_sim"], columns="vectorizer")


In [11]:
# res_table.drop("bleu", axis=0, level=1, inplace=True)

In [29]:
metrics = res_table.columns.get_level_values(0).unique()
for metric in metrics:
    display(res_table[metric].style.apply(highlight_max, axis = 1).apply(highlight_min, axis = 1).set_caption(metric.title().replace("_", " ")).format(precision=3))